<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/Encoder/fast_align_BPE_corpus%26vocab%26fertility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IWSLT

#### data load & prepare

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("bbaaaa/iwslt14-de-en-preprocess")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

## BPE

In [11]:
## 빠른 txt 작업

def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

In [4]:
en=[]
de=[]

for line in dataset['train']['translation']:
  en.append(line['en'])
  de.append(line['de'])

In [26]:
# corpus 준비

de_txt= '\n'.join(de)
to_txt(de_txt, 'de.txt') #20MB

en_txt= '\n'.join(en)
to_txt(en_txt, 'en.txt') #17MB

In [14]:
!pip install subword-nmt

#### de에 BPE 적용

참고:

원문으로 되돌리는법 text.replace('@@ ', '').replace('</w>', ' ').strip()

In [50]:
!subword-nmt learn-bpe -s 10000 < de.txt > bpe_de #bpe_de에 1000개의 merge(split) 규칙이 담김

100% 10000/10000 [00:45<00:00, 221.31it/s]


In [51]:
!subword-nmt apply-bpe -c bpe_de < de.txt > bpe_de.txt

In [52]:
bpe_de = read_txt('bpe_de.txt')

bpe_de[:1000]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein .\nund diese zwei zusammen zu bringen , erscheint vielleicht wie eine gewaltige aufgabe . aber was ich ihnen zu sagen versuche ist , dass es trotz dieser komplexität einige einfache themen gibt , von denen ich denke , wenn wir diese verstehen , können wir uns wirklich weiter entwickeln .\nund diese einfachen themen sind eigentlich keine komplexen wissenschaftlichen zusammen@@ hän@@ ge , sondern tat@@ sachen , die wir alle gut kennen .\nund ich werde mit dieser hier anfangen : wenn die mama nicht glücklich ist , ist keiner glücklich .\nkennen wir das nicht alle ? das haben wir alle schon erlebt .\nwenn wir das nehmen und darauf aufbauen , dann können wir einen schritt weiter gehen : wenn das meer nicht glücklich ist , ist keiner glücklich .\ndarum geht es in meinem vortrag .\nwir machen das meer auf viele verschiedene arten ziemlich un@@ glücklich .\ndas ist ein bild der can@@ ner@@ y ro@@ w von 19@@ 32 .\ndamals 

#### build_vocab

만들어버리고 load 하는게 편할듯

In [63]:
src_vocab = {'[PAD]': 0,'[UNK]': 1}
k=2

for token in bpe_de.split():
  if token not in src_vocab:
    src_vocab[token]= k
    k+=1

In [64]:
len(src_vocab)

10135

#### 영어도

In [58]:
!subword-nmt learn-bpe -s 10000 < en.txt > bpe_en

100% 10000/10000 [00:23<00:00, 422.08it/s]


In [59]:
!subword-nmt apply-bpe -c bpe_en < en.txt > bpe_en.txt

In [60]:
bpe_en = read_txt('bpe_en.txt')

bpe_en[:1000]

'and it can be a very complicated thing , what human health is .\nand bringing those two together might seem a very da@@ un@@ ting task , but what i &apos;m going to try to say is that even in that complexity , there &apos;s some simple themes that i think , if we understand , we can really move forward .\nand those simple themes aren &apos;t really themes about the complex science of what &apos;s going on , but things that we all pretty well know .\nand i &apos;m going to start with this one : if mom@@ ma ain &apos;t happy , ain &apos;t nobody happy .\nwe know that , right ? we &apos;ve experienced that .\nand if we just take that and we build from there , then we can go to the next step , which is that if the ocean ain &apos;t happy , ain &apos;t nobody happy .\nthat &apos;s the theme of my talk .\nand we &apos;re making the ocean pretty unhappy in a lot of different ways .\nthis is a shot of can@@ ner@@ y row in 19@@ 32 .\ncan@@ ner@@ y row , at the time , had the biggest industrial

In [62]:
tgt_vocab = {'[PAD]': 0,'[UNK]': 1}
k=2

for token in bpe_en.split():
  if token not in tgt_vocab:
    tgt_vocab[token]= k
    k+=1

In [65]:
len(tgt_vocab)

10010

#### save bpe vocabs

In [66]:
import pickle

with open('src_vocab.pkl', 'wb') as file:
    pickle.dump(src_vocab, file)

with open('tgt_vocab.pkl', 'wb') as file:
    pickle.dump(tgt_vocab, file)

## 각각 0.1MB

## fast align

In [71]:
bpe_de_split= bpe_de.split('\n')
bpe_en_split= bpe_en.split('\n')

## 길이 확인
len(bpe_de_split)==len(dataset['train']['translation'])==len(bpe_en_split)

True

In [72]:
bpe_de_split[0]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein .'

In [80]:
## fast align input 전처리

def for_fastalign(bpe_de_split, bpe_en_split):
  for_align= []

  for de, en in zip(bpe_de_split, bpe_en_split):
    for_align.append(de+' ||| '+en)

  return '\n'.join(for_align)

In [81]:
fastalign_input= for_fastalign(bpe_de_split, bpe_en_split)

In [83]:
to_txt(fastalign_input, 'train.txt')

In [82]:
fastalign_input[:1000]

'und was menschliche gesundheit ist , kann auch ziemlich kompliziert sein . ||| and it can be a very complicated thing , what human health is .\nund diese zwei zusammen zu bringen , erscheint vielleicht wie eine gewaltige aufgabe . aber was ich ihnen zu sagen versuche ist , dass es trotz dieser komplexität einige einfache themen gibt , von denen ich denke , wenn wir diese verstehen , können wir uns wirklich weiter entwickeln . ||| and bringing those two together might seem a very da@@ un@@ ting task , but what i &apos;m going to try to say is that even in that complexity , there &apos;s some simple themes that i think , if we understand , we can really move forward .\nund diese einfachen themen sind eigentlich keine komplexen wissenschaftlichen zusammen@@ hän@@ ge , sondern tat@@ sachen , die wir alle gut kennen . ||| and those simple themes aren &apos;t really themes about the complex science of what &apos;s going on , but things that we all pretty well know .\nund ich werde mit diese

In [6]:
%cd /content/
!git clone https://github.com/clab/fast_align.git
!sudo apt-get install libgoogle-perftools-dev libsparsehash-dev
%cd fast_align
!mkdir build
%cd build
!cmake ..
!make
%cd /content/

/content
Cloning into 'fast_align'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 213 (delta 2), reused 4 (delta 2), pack-reused 204
Receiving objects: 100% (213/213), 70.68 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (110/110), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libsparsehash-dev libunwind-dev
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,424 kB of archives.
After this operation, 10.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunwind-dev amd64 1.3.2-2build2.1 [1,883 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgoogle-perftools-dev amd64 2.9.1-0ubuntu3 [470 kB]
Get

In [8]:
## ✰✰✰✰✰ .align 읽어서 feritility label로 변환 ✰✰✰✰✰

from collections import Counter

def read_align(name):
  counts= []

  with open(name, 'r') as file:
      for line in file:
          line = line.strip()

          #pairs: 'a list' of 'n-n's
          pairs = line.split()

          #left_numbers: ['0', '0', '2', '1', ...
          left_numbers = [pair.split('-')[0] for pair in pairs]

          #Counter({'14': 5, '3': 4, '2': 3,...
          count = Counter(left_numbers)

          #count: [1, 1, 3, 4, 1, 0, 0, 0, ...
          count= [count.get(str(i), 0) for i in range(max(map(int, count.keys())) + 1)]
          counts.append(count)

          ## count sum check (다르면 fertility label이 될수 없음)
          if len(left_numbers)!=sum(count):
            print('what!?', len(left_numbers),sum(count))
            break

      return counts

#### fertility label 생성

In [84]:
## txt 파일 하나에 적용 (.txt와 .align 이름만 수정)
!/content/fast_align/build/fast_align -i train.txt -d -o -v > train.align

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
expected target length = source length * 0.99199
ITERATION 1
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -7.39955e+07
  log_2 likelihood: -1.06753e+08
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.168268
       size counts: 3190
ITERATION 2
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -1.99355e+07
  log_2 likelihood: -2.87608e+07
     cross entropy: 8.05478
        perplexity: 265.9

(Without BPE) 기본적으로 final perplexity가 40 정도인데 괜찮은 것인가..?

(With BPE) 와 첫 perplexity가 1e+09 ㅋㅋㅋㅋ final은 58인데??

In [85]:
## fast align을 다 돌렸다면 여기로
train_fertility= read_align('train.align')

#### fertility check example

wir -> 1 단어 (we)

durchlaufen -> 2단어 (go through)

initiatiohnsrituale -> 2단어 (initiation rites)

. -> 1단어(.)

In [86]:
train_fertility[4]

[1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 1, 1, 1]

#### export (pickle)

In [87]:
import pickle

with open('train_ft_bpe.pkl', 'wb') as file: #8MB
    pickle.dump(train_fertility, file)